In [2]:
import yfinance as yf

bank_nifty = yf.Ticker("^NSEBANK")

In [3]:
data = bank_nifty.history(period="max", interval="5m", start="2023-12-08")
close = data['Close'].to_numpy()
PERIOD_1 = 12
PERIOD_2 = 26

from openalerts.core import macd
close, mov_1, mov_2 = macd(close, PERIOD_1, PERIOD_2)

import matplotlib.pyplot as plt

time_whole_day = int(60*6.25/5)

date_time = data.index[-len(mov_2):]
plt.plot(date_time[-2*time_whole_day: -time_whole_day], close[-2*time_whole_day: -time_whole_day])
plt.plot(date_time[-2*time_whole_day: -time_whole_day], mov_1[-2*time_whole_day: -time_whole_day], color='green')
plt.plot(date_time[-2*time_whole_day: -time_whole_day], mov_2[-2*time_whole_day: -time_whole_day], color='red')

plt.show()
print(close[-1])

ModuleNotFoundError: No module named 'openalerts'

In [ ]:
""""
Ultimate Oscillator
Source: http://stockcharts.com/school/doku.php?id=chart_school:technical_indicators:ultimate_oscillator
Params: 
    data: pandas DataFrame
	period_1: the period of the first average (7 days recommended)
	period_2: the period of the second average (14 days recommended)
	period_3: the period of the third average (28 days recommended)
	high_col: the name of the HIGH values column
	low_col: the name of the LOW values column
	close_col: the name of the CLOSE values column
    
Returns:
    copy of 'data' DataFrame with 'ultimate_oscillator' column added
"""
def ultimate_oscillator(data, period_1=7,period_2=14, period_3=28, high_col='High', low_col='Low', close_col='Close'):
    data['ultimate_oscillator'] = 0.
    data['uo_bp'] = 0.
    data['uo_tr'] = 0.
    data['uo_avg_1'] = 0.
    data['uo_avg_2'] = 0.
    data['uo_avg_3'] = 0.

    for index,row in data.iterrows():
        if index > 0:
                           
            bp = row[close_col] - min(row[low_col], data.at[index-1, close_col])
            tr = max(row[high_col], data.at[index-1, close_col]) - min(row[low_col], data.at[index-1, close_col])
            
            data.set_value(index, 'uo_bp', bp)
            data.set_value(index, 'uo_tr', tr)
            if index >= period_1:
                uo_avg_1 = sum(data['uo_bp'][index-period_1:index]) / sum(data['uo_tr'][index-period_1:index])
                data.set_value(index, 'uo_avg_1', uo_avg_1)
            if index >= period_2:
                uo_avg_2 = sum(data['uo_bp'][index-period_2:index]) / sum(data['uo_tr'][index-period_2:index])
                data.set_value(index, 'uo_avg_2', uo_avg_2)
            if index >= period_3:
                uo_avg_3 = sum(data['uo_bp'][index-period_3:index]) / sum(data['uo_tr'][index-period_3:index])
                data.set_value(index, 'uo_avg_3', uo_avg_3)
                uo = (4 * uo_avg_1 + 2 * uo_avg_2 + uo_avg_3) / 7
                data.set_value(index, 'ultimate_oscillator', uo)

    data = data.drop(['uo_bp', 'uo_tr', 'uo_avg_1', 'uo_avg_2', 'uo_avg_3'], axis=1)
        
    return data   